In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
import time
import json

import subprocess
from subprocess import PIPE, Popen
import os
from tqdm import tqdm
import re

word2abstract = {}
def worker(entity_name, i):
    global word2abstract
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query2 = '''
    SELECT ?object 
    WHERE {dbr:''' + entity_name + '''  dbo:abstract  ?object }'''

    sparql.setQuery(query2)  # the previous query as a literal string
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(2)

    tmp = sparql.query()
    one_res = tmp.convert()

    #if one_res['results']['bindings'] == []:
    
    
    for result in one_res['results']['bindings']:

        lang,value = result['object']['xml:lang'], result['object']['value']

        value = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", "", value)
        if lang == 'en':
            return entity_name, value
        

def collectMyResult(result):
    global word2abstract
    # if result:
    try:
        word2abstract[result[0]] = result[1]
        print("Got result {}".format(result[0]))
    except Exception as e:
        print('error:', e)
def abortable_worker(func, *args, **kwargs):
    timeout = 30
    p = ThreadPool(1)
    res = p.apply_async(func, args=args)
    try:
        out = res.get(timeout)  # Wait timeout seconds for func to complete.
        return out
    except multiprocessing.TimeoutError:
        print("Aborting due to timeout")
        raise

if __name__ == "__main__":
    word2abstract = {}
    with open("./need_abstract.json","r",encoding='utf-8') as f: # entities need abstract
        need_abstract = json.load(f)
    tic = time.time()
    pool = multiprocessing.Pool(processes=60,maxtasksperchild=1)

    featureClass = [[need_abstract[i], i] for i in tqdm(range(0,len(need_abstract),1))] #list of arguments
    for f in featureClass:
        abortable_func = partial(abortable_worker, worker, timeout=30)
        pool.apply_async(abortable_func, args=f,callback=collectMyResult)
    pool.close()
    pool.join()
    
    toc = time.time()
    shijian = toc-tic
    print("time consuming:",shijian)
    
    with open("./word2abstract.json","w",encoding='utf-8') as f:
            json.dump(word2abstract,f)
    print(len(word2abstract))